In [ ]:
import numpy as np
import json
import pandas as pd
from pandas.io.json import json_normalize
import simplejson as json

In [ ]:
import glob
import os

path = 'C://Users/Tom/Documents/Kickstarter/JSONs' 
list_files = os.listdir(path)
files = []

for file in list_files:
    file = path + '/' + file
    files.append(file)
print(file)

In [ ]:
filenum = 0
for file in files:
    chunks = []
    reader = pd.read_json(file, lines=True, chunksize=4096)   

    for chunk in reader:
        chunks.append(chunk['data'])
        
    normalized = []
    
    for chunk in range(len(chunks)):
        normalized.append(json_normalize(chunks[chunk]))
        normalized[chunk] = normalized[chunk][['id', 'backers_count', 'blurb', 'category.name', 'category.slug', 'country', 'currency',
                                                  'goal', 'launched_at', 'deadline', 'location.displayable_name', 'location.country', 'location.state',
                                                  'location.type', 'name', 'usd_pledged', 'slug', 'spotlight', 'staff_pick', 'static_usd_rate', 'state']]
    master = pd.concat(normalized, sort=False)
    new_filename = 'file' + str(filenum) + '.csv'
    filenum += 1
    master.to_csv(new_filename)


In [ ]:
master = pd.DataFrame(master)

In [ ]:
# Feature Engineering and Cleaning Columns
import datetime
master['launched_at'] = master['launched_at'].apply(datetime.datetime.utcfromtimestamp)
master['deadline'] = master['deadline'].apply(datetime.datetime.utcfromtimestamp)


In [ ]:
master['year'] = master['launched_at'].apply(lambda x: datetime.date.timetuple(x)[0])
master['month'] =  master['launched_at'].apply(lambda x: datetime.date.timetuple(x)[1])
master['day'] = master['launched_at'].apply(lambda x: datetime.date.timetuple(x)[2])
master['hour'] = master['launched_at'].apply(lambda x: datetime.date.timetuple(x)[3])

In [ ]:
master['days_to_deadline'] = (master['deadline'] - master['launched_at']).apply(lambda x: x/np.timedelta64(1,'D'))
master['days_to_deadline'] = master['days_to_deadline'].apply(lambda x: format(x, '.0f'))

In [ ]:
# Feature engineering: changing goals to constant currency and rounding to 2 decimal places
master['goal_USD'] = master['goal'] * master['static_usd_rate']

In [ ]:
master['category_name'] = master['category.name']
master['category_slug'] = master['category.slug'].apply(lambda x: x.split('/')[0])
master = master.drop(['category.name', 'category.slug'], axis=1)

In [ ]:
# Changing object data types to str
[master[column].apply(lambda x: str(x)) for column in 
['id', 'blurb', 'country', 'currency', 'location.displayable_name', 'location.country', 'location.state', 
 'location.type', 'name', 'slug', 'state', 'category_name', 'category_slug']]; 

In [ ]:
# Adding blurb length to model
def word_count(string):
    tokens = string.split()
    n_tokens = len(tokens)
    return(n_tokens)

master['blurb_length'] = master['blurb'].apply(lambda x: word_count(str(x)))

In [ ]:
master.isnull().sum()

In [ ]:
master[master['country'] != master['location.country']]

In [ ]:
master['location_type'] = master['location.type'].fillna('Miscellaneous')
master['location_country'] = master['location.country'].fillna(master['country'])
master['location_state'] = master['location.state'].fillna('-')
master['blurb'] = master['blurb'].fillna('-')
master['location_displayable_name'] = master['location.displayable_name'].fillna('-')

In [ ]:
master = master.drop(['country', 'location.type', 'location.state', 'location.displayable_name'], axis=1)

In [ ]:
# df['new column name'] = df['df column_name'].apply(lambda x: 'value if condition is met' if x condition else 'value if condition is not met')
def binary_state(x):
    if x['usd_pledged'] < x['goal_USD']:
        return 'failed'
    else: return 'successful'
master['binary_state'] = master.apply(lambda x: binary_state(x), axis=1)    

In [ ]:
master.to_csv('kickstarter.csv')